In [1]:
%load_ext autoreload
%autoreload 2

The autoreload module is not an IPython extension.


UsageError: Line magic function `%autoreload` not found.


In [2]:
from file_reader.preprocessing import YoubikeDataReader, YoubikeDataFactory
import filters.NTU
import filters.time_filter

Weekday = filters.time_filter.Weekday

file_location = r"DATA/202209_YouBike2.0票證刷卡資料.csv"

youbike_reader = YoubikeDataReader(file_location)


In [3]:
location_filter = filters.NTU.NTU_location_filter
upper_interval_filter = filters.time_filter.over_interval(mins= 30)
lower_interval_filter = filters.time_filter.below_interval(hours=2)
weekend = filters.time_filter.weekday_filter([Weekday.SAT, Weekday.SUN])

youbike_df_factory = (
                        YoubikeDataFactory(youbike_reader)
                        .filter_location(location_filter)
                        .filter_time(upper_interval_filter)
                        .filter_time(lower_interval_filter)
                        .filter_time(weekend)
                    )
ntu_df = youbike_df_factory.df

In [4]:
grouped = ntu_df.groupby(['rent_station','return_station']).agg({'rent': ['count', 'sum', 'max', 'min']})
grouped.columns = grouped.columns.map('_'.join)
grouped = grouped.sort_values(by = ('rent_count'), ascending=False).reset_index()

In [5]:
from geo import add_coordinate
geo_data = add_coordinate(grouped)
geo_data = geo_data[['rent_station', 'return_station', 'rent_count', 'lat_rent', 'lng_rent', 'lat_return', 'lng_return']]

In [6]:
geo_data

,rent_station,return_station,rent_count,lat_rent,lng_rent,lat_return,lng_return
0,臺大男一舍前,臺大男一舍前,43,25.01637,121.54535,25.01637,121.54535
1,臺大第一活動中心西南側,臺大男一舍前,1,25.01761,121.53995,25.01637,121.54535
2,臺大男六舍前,臺大男一舍前,2,25.01664,121.54520,25.01637,121.54535
3,臺大男七舍前,臺大男一舍前,1,25.01537,121.54698,25.01637,121.54535
4,臺大小福樓東側,臺大男一舍前,4,25.01846,121.53760,25.01637,121.54535
...,...,...,...,...,...,...,...
501,臺大舊體育館西側,臺大公館停車場西北側,1,25.01980,121.53583,25.01345,121.53798
502,臺大明達館北側(員工宿舍),臺大明達館北側(員工宿舍),2,25.01816,121.54469,25.01816,121.54469
503,臺大博理館西側,臺大博雅館西側,1,25.01922,121.54195,25.01890,121.53600
504,臺大天文數學館南側,臺大思亮館東南側,1,25.02108,121.53783,25.02074,121.53906


In [7]:
from geo.presenter import MapEdgePresentation
import folium

In [8]:
map_center = [25.0169457,121.5385459] # Set the center of the map
zoom_level = 15 # Set the zoom level of the map
m = folium.Map(location=map_center, zoom_start=zoom_level)

presenter = MapEdgePresentation(geo_data, m).set_edge_weight('rent_count')

In [9]:
presenter.present()